In [ ]:
# !sudo apt-get install unzip
# !unzip Songs.zip -d Songs

In [ ]:
# !unzip Wav_Aravind.zip -d .

In [ ]:
from tqdm import tqdm
import pandas as pd
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import os
import time
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import subprocess
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/',force_remount=True)
%cd /content/gdrive/MyDrive/Download\ Audio/

Mounted at /content/gdrive/
/content/gdrive/MyDrive/Download Audio


In [ ]:
error_indices=[]

In [ ]:
wav=set([i.split(".")[0] for i in os.listdir("Wav/")])
songs=set([i.split(".")[0] for i in os.listdir("Songs/")])

In [ ]:
# wav - songs
songs - wav

set()

In [ ]:
# for i in os.listdir("Wav/"):
#     if re.search('mp4', i):
#         os.remove("Wav/"+i)

In [ ]:
len(os.listdir("Songs/"))

10099

In [ ]:
done_wavs = set([i.split(".")[0] for i in os.listdir("Wav")])

In [ ]:
error_indices=[]
count=0
mel_shape=[]
short_files=[]
for path in tqdm(sorted(os.listdir("Songs/"))[:4000],ncols=100):
    count+=1
    # if count==51:
    #     break
    try:
        name=path.split(".")[0]
        # if name in done_wavs:
        #     continue
        command2mp3 = "ffmpeg -i Songs/"+path+" "+name+".mp3"
        command2wav = "ffmpeg -i "+name+".mp3 "+"Wav/"+name+".wav"
        os.system(command2mp3)
        os.system(command2wav)
        os.remove(name+".mp3")

    except:
        error_indices.append(path)
        with open("error_log_mel.txt", "a") as fp: 
            fp.write(str(path)+"\n")

In [ ]:
!pip install torchaudio

     |████████████████████████████████| 2.9 MB 13.7 MB/s 


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from functools import partial
import math
import heapq
import tensorflow as tf
from torchaudio.transforms import MelScale, Spectrogram
from glob import glob
torch.set_default_tensor_type('torch.cuda.FloatTensor')
import numpy as np
import os

OSError: ignored

In [ ]:
torch.version.cuda

'11.1'

In [ ]:
#@title Hyperparameters 
learning_rate = 0.0005 #@param {type:"raw"}
num_epochs_to_train =  100#@param {type:"integer"}
batch_size =  32#@param {type:"integer"}
vector_dimension = 64 #@param {type:"integer"}

hop=256               #hop size (window size = 4*hop)
sr=44100              #sampling rate
min_level_db=-100     #reference values to normalize data
ref_level_db=20

LEARNING_RATE = learning_rate
BATCH_SIZE = batch_size
EPOCHS = num_epochs_to_train
VECTOR_DIM=vector_dimension

shape=128           #length of time axis of split specrograms         
spec_split=1

In [ ]:
specobj = Spectrogram(n_fft=4*hop, win_length=4*hop, hop_length=hop, pad=0, power=2, normalized=False)
specfunc = specobj.forward
melobj = MelScale(n_mels=hop, sample_rate=sr, f_min=0.)
melfunc = melobj.forward

In [ ]:
def melspecfunc(waveform):
  specgram = specfunc(waveform)
  mel_specgram = melfunc(specgram)
  return mel_specgram

In [ ]:
def audio_array(path,part = None,tot = None,song_path = None):
  if song_path != None:
      path = "Wav/"+song_path
      x, sr = tf.audio.decode_wav(tf.io.read_file(path), 1)
      x = np.array(x, dtype=np.float32)
      return np.array([x])
  ls = glob(f'{path}/*.wav')
  adata = []
  for i in tqdm(range(part*len(ls)//tot,(part+1)*len(ls)//tot), ncols = 100):
    x, sr = tf.audio.decode_wav(tf.io.read_file(ls[i]), 1)
    x = np.array(x, dtype=np.float32)
    adata.append(x)
  return np.array(adata)

In [ ]:
def tospec(data):
  specs=np.empty(data.shape[0], dtype=object)
  for i in range(data.shape[0]):
    x = data[i]
    S=prep(x)
    S = np.array(S, dtype=np.float32)
    specs[i]=np.expand_dims(S, -1)
#   print(specs.shape)
  return specs

In [ ]:
def prep(wv,hop=192):
  S = np.array(torch.squeeze(melspecfunc(torch.Tensor(wv).view(1,-1))).detach().cpu())
  S = librosa.power_to_db(S)-ref_level_db
  return normalize(S)

In [ ]:
def normalize(S):
  return np.clip((((S - min_level_db) / -min_level_db)*2.)-1., -1, 1)  

In [ ]:
short_audios = []

In [ ]:
undone = ["SOFZUKC12AC468D4DF.wav","SOFZSAT12AF72A0806.wav","SOFZXCH12A58A7B467.wav"]

In [ ]:
audio_directory = "Wav/"
already_npy = set([i.split(".")[0] for i in os.listdir("Single_NPYs")])
for i in tqdm(sorted(os.listdir(audio_directory)),ncols = 100):
    if i.split(".")[0] in already_npy:
        continue
    awv = audio_array(audio_directory,song_path=i)
    aspec = tospec(awv)
    x_train = aspec[0][:,:4096,:]
    if x_train.shape[1]<4096:
        print("Error in",i,(x_train.shape))
        short_audios.append(i)
        continue
    np.save("Single_NPYs/"+i.split(".")[0], x_train)

100%|████████████████████████████████████████████████████| 10099/10099 [00:00<00:00, 1156066.49it/s]


In [ ]:
tot = 40
audio_directory = "Wav/"
for part in range(38,tot):
    print("Starting",part+1,"out of", tot)
    awv = audio_array(audio_directory,part = part,tot = tot)
    aspec = tospec(awv)  
    x_train = np.zeros((aspec.shape[0],256,4096,1))
    # x_train = np.array([x[:, :4096, :] for x in aspec])
    count = 0
    for i in range(aspec.shape[0]):
        try:
            x_train[count] = aspec[i][:, :4096, :]
            count += 1
        except:
            print("Eroror in",i)
            print(aspec[i][:256, :4096, :1].shape,"\t", x_train[i].shape)

    x_train=x_train[:count,:,:,:]
    print(count,x_train.shape)
    pat = str(part+1)+"by"+str(tot)
    # if pat+".npy" in os.listdir("NPY"):
    #     print("Rename!",pat)
    np.save("NPY/"+pat, x_train)

In [ ]:
len(os.listdir("/content/gdrive/MyDrive/Download Audio/Wav"))

10099